# Llama2-7B-LoRA-8bit

QLoRA：NF4量化、双重量化、分页优化器

加载显存4.486G，训练显存7.534G（batchsize为1且不开梯度检查点时）

## Step1 导入相关包

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "5"

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

/node6_1/tanshuai/.conda/envs/abc/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step2 加载数据集

In [2]:
ds = load_dataset("lifefabric/alpaca_data_cleaned.zh", split="train[:10000]")
ds

Repo card metadata block was not found. Setting CardData to empty.


Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 10000
})

In [ ]:
ds[:3]

## Step3 数据集预处理

In [3]:
tokenizer = AutoTokenizer.from_pretrained("/node6_1/tanshuai/modelscope/Llama-2-7b-ms")
tokenizer

LlamaTokenizerFast(name_or_path='/node6_1/tanshuai/modelscope/Llama-2-7b-ms', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	32000: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
}

In [ ]:
tokenizer.pad_token_id = 2

In [4]:
def process_func(example):
    MAX_LENGTH = 512
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer("\n".join(["Human: " + example["instruction"], example["input"]]).strip() + "\n\nAssistant: ", add_special_tokens=False)
    response = tokenizer(example["output"], add_special_tokens=False)
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.eos_token_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.eos_token_id]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [5]:
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)
tokenized_ds

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 10000
})

In [ ]:
print(tokenized_ds[0]["input_ids"])

In [ ]:
# tokenizer.decode(tokenized_ds[0]["input_ids"])

In [ ]:
tokenizer(tokenizer.eos_token)

In [ ]:
# tokenizer('abc' + tokenizer.eos_token)

In [ ]:
# Llama的词表没针对中文设计，一个字解码出多个tokens
tokenizer("呀", add_special_tokens=False)

In [ ]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_ds[1]["labels"])))

## Step4 创建模型

In [6]:
import torch
# 多卡情况，可以去掉device_map="auto"，否则会将模型拆开
model = AutoModelForCausalLM.from_pretrained("/node6_1/tanshuai/modelscope/Llama-2-7b-ms", 
                                             low_cpu_mem_usage=True, 
                                             torch_dtype=torch.bfloat16, 
                                             device_map="auto",
                                             load_in_4bit=True,
                                             bnb_4bit_compute_dtype=torch.bfloat16,
                                             bnb_4bit_quant_type="nf4",
                                             bnb_4bit_use_double_quant=True)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 2/2 [00:10<00:00,  5.33s/it]
/node6_1/tanshuai/.conda/envs/abc/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/node6_1/tanshuai/.conda/envs/abc/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation 

In [7]:
model.dtype

torch.bfloat16

In [8]:
for name, parameter in model.named_parameters():
    print(name, parameter.dtype)

model.embed_tokens.weight torch.bfloat16
model.layers.0.self_attn.q_proj.weight torch.uint8
model.layers.0.self_attn.k_proj.weight torch.uint8
model.layers.0.self_attn.v_proj.weight torch.uint8
model.layers.0.self_attn.o_proj.weight torch.uint8
model.layers.0.mlp.gate_proj.weight torch.uint8
model.layers.0.mlp.up_proj.weight torch.uint8
model.layers.0.mlp.down_proj.weight torch.uint8
model.layers.0.input_layernorm.weight torch.bfloat16
model.layers.0.post_attention_layernorm.weight torch.bfloat16
model.layers.1.self_attn.q_proj.weight torch.uint8
model.layers.1.self_attn.k_proj.weight torch.uint8
model.layers.1.self_attn.v_proj.weight torch.uint8
model.layers.1.self_attn.o_proj.weight torch.uint8
model.layers.1.mlp.gate_proj.weight torch.uint8
model.layers.1.mlp.up_proj.weight torch.uint8
model.layers.1.mlp.down_proj.weight torch.uint8
model.layers.1.input_layernorm.weight torch.bfloat16
model.layers.1.post_attention_layernorm.weight torch.bfloat16
model.layers.2.self_attn.q_proj.weigh

In [ ]:
model.config

## LoRA

### PEFT 1 配置文件

In [9]:
from peft import LoraConfig, get_peft_model, TaskType

config = LoraConfig(task_type=TaskType.CAUSAL_LM)
print(config)

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules=None, lora_alpha=8, lora_dropout=0.0, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False))


### PEFT 2 创建模型

In [10]:
model = get_peft_model(model, config)

In [ ]:
print(config)

In [11]:
model.print_trainable_parameters()

trainable params: 4,194,304 || all params: 6,742,609,920 || trainable%: 0.0622


In [ ]:
for name, parameter in model.named_parameters():
    print(name, parameter.dtype)

In [ ]:
# model.half()

In [ ]:
for name, parameter in model.named_parameters():
    print(name, parameter.dtype)

## Step5 配置训练参数

In [12]:
args = TrainingArguments(
    output_dir="./chatbot",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    logging_steps=10,
    num_train_epochs=1,
    # gradient_checkpointing=True,
    optim="paged_adamw_32bit"  # 分页优化器
)

In [13]:
trainer = Trainer(
    model=model,
    args=args,
    tokenizer=tokenizer,
    train_dataset=tokenized_ds.select(range(5000)),
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True)
)

/tmp/ipykernel_3718822/903977660.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


## Step7 模型训练

In [14]:
trainer.train()

Step,Training Loss
10,1.391000
20,1.285600
30,1.310100
40,1.223000
50,1.261500
60,1.175600
70,1.159500
80,1.140500
90,1.161900
100,1.089200


TrainOutput(global_step=625, training_loss=1.1151125839233398, metrics={'train_runtime': 1061.9484, 'train_samples_per_second': 4.708, 'train_steps_per_second': 0.589, 'total_flos': 5.467673099845632e+16, 'train_loss': 1.1151125839233398, 'epoch': 1.0})

## Step8 模型推理

In [15]:
# 生成时间明显变长
model.eval()
ipt = tokenizer("Human: {}\n{}".format("考试有什么技巧？", "").strip() + "\n\nAssistant: ", return_tensors="pt").to(model.device)
tokenizer.decode(model.generate(**ipt, max_length=128, do_sample=True, eos_token_id=tokenizer.eos_token_id)[0], skip_special_tokens=True)

'Human: 考试有什么技巧？\n\nAssistant:  考试有很多技巧, 但是一个关键的技巧是准备好。 在考试之前, 你必须确定你所要做的内容, 包括所有必要的知识和技能。 你还必须确定你的时间, 以便'

In [16]:
# 最好不要合并，由于量化过程中的四舍五入误差，可能会导致生成的结果与不合并时有所不同。
model.merge_and_unload()

/node6_1/tanshuai/.conda/envs/abc/lib/python3.9/site-packages/peft/tuners/lora/bnb.py:336: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNor

In [17]:
for name, parameter in model.named_parameters():
    print(name, parameter.dtype)

base_model.model.model.embed_tokens.weight torch.bfloat16
base_model.model.model.layers.0.self_attn.q_proj.weight torch.uint8
base_model.model.model.layers.0.self_attn.k_proj.weight torch.uint8
base_model.model.model.layers.0.self_attn.v_proj.weight torch.uint8
base_model.model.model.layers.0.self_attn.o_proj.weight torch.uint8
base_model.model.model.layers.0.mlp.gate_proj.weight torch.uint8
base_model.model.model.layers.0.mlp.up_proj.weight torch.uint8
base_model.model.model.layers.0.mlp.down_proj.weight torch.uint8
base_model.model.model.layers.0.input_layernorm.weight torch.bfloat16
base_model.model.model.layers.0.post_attention_layernorm.weight torch.bfloat16
base_model.model.model.layers.1.self_attn.q_proj.weight torch.uint8
base_model.model.model.layers.1.self_attn.k_proj.weight torch.uint8
base_model.model.model.layers.1.self_attn.v_proj.weight torch.uint8
base_model.model.model.layers.1.self_attn.o_proj.weight torch.uint8
base_model.model.model.layers.1.mlp.gate_proj.weight tor